In [ ]:
import os
import h5py
import argparse
import numpy as np
from tqdm import tqdm
from PIL import Image
import torch
from torchvision import transforms
from models.csrnet_tinyvgg import TinyCSRNet
from models.csrnet_mbv3 import MobileCSRNet
from models.csrnet_vgg import CSRNet

# build/csrnet_vgg_B.pth
# Evaluation Results:
#   MAE  = 20.31
#   RMSE = 37.47

# build/best_kd_B_mobilenet_final.pth
# Evaluation Results:
#   MAE  = 23.37
#   RMSE = 37.00

MODEL="build/csrnet_vgg_A.pth"
PREDDIR=""
PART="A"

def load_model(model_path, device):
    model = CSRNet().to(device)
    # model.load_state_dict(torch.load(MODEL, map_location=device, weights_only=False))
    checkpoint = torch.load(MODEL, map_location=device, weights_only=False)
    model.load_state_dict(checkpoint['state_dict'])
    model.eval()
    return model

def get_transform():
    return transforms.Compose([
        # transforms.Resize((512, 512)),
        # transforms.RandomHorizontalFlip(p=0.5),
        # transforms.ColorJitter(brightness=0.2, contrast=0.2),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])

def predict_density(model, image_path, device):
    image = Image.open(image_path).convert("RGB")
    input_tensor = get_transform()(image).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(input_tensor).cpu().squeeze(0).squeeze(0).numpy()
    return output

def evaluate(gt_dir, img_dir=None, model=None, pred_dir=None, device='cpu'):
    mae, mse, total = 0.0, 0.0, 0
    gt_files = sorted([f for f in os.listdir(gt_dir) if f.endswith('.h5')])

    for fname in tqdm(gt_files, desc="Evaluating"):
        gt_path = os.path.join(gt_dir, fname)
        with h5py.File(gt_path, 'r') as f:
            gt_density = np.asarray(f['density'])
        gt_count = gt_density.sum()

        if model and img_dir:
            img_path = os.path.join(img_dir, fname.replace('.h5', '.jpg').replace('GT_', ''))
            pred_density = predict_density(model, img_path, device)
        elif pred_dir:
            pred_path = os.path.join(pred_dir, fname)
            if not os.path.exists(pred_path):
                print(f"Missing prediction for {fname}, skipping.")
                continue
            with h5py.File(pred_path, 'r') as f:
                pred_density = np.asarray(f['density'])
        else:
            raise ValueError("Either model+img_dir or pred_dir must be provided.")

        pred_count = pred_density.sum()
        # print(f"GT {gt_count:.2f}\t Predict {pred_count:.2f}")
        
        error = abs(gt_count - pred_count)
        mae += error
        mse += error ** 2
        total += 1

    mae /= total
    rmse = (mse / total) ** 0.5

    print(f"\nEvaluation Results:")
    print(f"  MAE  = {mae:.2f}")
    print(f"  RMSE = {rmse:.2f}")

def main():
    part = PART.upper()
    root = os.path.abspath("dataset")
    gt_dir = os.path.join(root, f"part_{part}", "test_data", "ground-truth")
    img_dir = os.path.join(root, f"part_{part}", "test_data", "images")

    if not os.path.isdir(gt_dir):
        raise FileNotFoundError(f"Ground truth directory not found: {gt_dir}")

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    if MODEL:
        model = load_model(MODEL, device)
        evaluate(gt_dir, img_dir=img_dir, model=model, device=device)
    elif PREDDIR:
        pred_dir = os.path.abspath(PREDDIR)
        evaluate(gt_dir, pred_dir=pred_dir)
    else:
        raise ValueError("You must provide either --model or --pred_dir")

if __name__ == '__main__':
    main()


Evaluating:   2%|▏         | 4/182 [00:00<00:05, 29.80it/s]

GT 172.00	 Predict 259.14
GT 502.00	 Predict 377.42
GT 389.00	 Predict 366.59
GT 211.00	 Predict 208.89
GT 223.00	 Predict 244.75
GT 430.00	 Predict 450.88
GT 1174.00	 Predict 1304.84


Evaluating:   6%|▌         | 11/182 [00:00<00:06, 25.96it/s]

GT 265.00	 Predict 210.70
GT 1232.00	 Predict 1178.56
GT 289.00	 Predict 195.10
GT 181.00	 Predict 179.15
GT 379.00	 Predict 367.04


Evaluating:   9%|▉         | 17/182 [00:00<00:06, 25.53it/s]

...

GT 579.00	 Predict 537.93
GT 417.00	 Predict 392.92
GT 86.00	 Predict 83.82
GT 382.00	 Predict 425.22
GT 371.00	 Predict 273.23
GT 2256.00	 Predict 1408.64


Evaluating:  99%|█████████▉| 180/182 [00:05<00:00, 32.21it/s]

GT 101.00	 Predict 101.22
GT 1365.00	 Predict 1281.41
GT 255.00	 Predict 220.43
GT 69.00	 Predict 56.85
GT 190.00	 Predict 141.85
GT 246.00	 Predict 214.27
GT 477.00	 Predict 581.72


Evaluating: 100%|██████████| 182/182 [00:05<00:00, 30.84it/s]

GT 153.00	 Predict 160.43
GT 242.00	 Predict 148.56

Evaluation Results:
  MAE  = 84.76
  RMSE = 157.12


In [3]:
import torch
from models.csrnet_mbv3 import MobileCSRNet
from fvcore.nn import FlopCountAnalysis, parameter_count

MODEL="build/csrnet_mobile_A.pt"

# Inisialisasi model
model = MobileCSRNet()
model.load_state_dict(torch.load(MODEL, map_location='cpu', weights_only=False))
# checkpoint = torch.load(MODEL, map_location=device, weights_only=False)
# model.load_state_dict(checkpoint['state_dict'])
model.eval()

# Input dummy sesuai resolusi saat training
dummy_input = torch.randn(1, 3, 512, 512)

# Hitung FLOPs dan jumlah parameter
flops = FlopCountAnalysis(model, dummy_input)
params = parameter_count(model)

print(f"FLOPs: {flops.total() / 1e9:.2f} GFLOPs")
print(f"Params: {params[''] / 1e6:.2f} M")

Unsupported operator aten::hardswish_ encountered 20 time(s)
Unsupported operator aten::add_ encountered 10 time(s)
Unsupported operator aten::hardsigmoid encountered 8 time(s)
Unsupported operator aten::mul encountered 8 time(s)


FLOPs: 3.90 GFLOPs
Params: 13.66 M


In [18]:
import os
import h5py
import argparse
import numpy as np
from tqdm import tqdm
from PIL import Image
import torch
import torchvision.transforms.functional as F
from torchvision import transforms
from models.csrnet_tinyvgg import TinyCSRNet
from models.csrnet_mbv3 import MobileCSRNet
from models.csrnet_vgg import CSRNet

# build/csrnet_vgg_B.pth
# Evaluation Results:
#   MAE  = 20.31
#   RMSE = 37.47

# build/best_kd_B_mobilenet_final.pth
# Evaluation Results:
#   MAE  = 23.37
#   RMSE = 37.00

MODEL="build/csrnet_vgg_B.pth"
PREDDIR=""
PART="A"

def load_model(model_path, device):
    model = CSRNet().to(device)
    # model.load_state_dict(torch.load(MODEL, map_location=device, weights_only=False))
    checkpoint = torch.load(MODEL, map_location=device, weights_only=False)
    model.load_state_dict(checkpoint['state_dict'])
    model.eval()
    return model

def get_transform():
    return transforms.Compose([
        # transforms.Resize((512, 512)),
        # transforms.RandomHorizontalFlip(p=0.5),
        # transforms.ColorJitter(brightness=0.2, contrast=0.2),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])

def predict_density(model, image_path, device):
    image = Image.open(image_path).convert("RGB")
    # image = 255.0 * F.to_tensor(image)
    # image[0,:,:]=image[0,:,:]-92.8207477031
    # image[1,:,:]=image[1,:,:]-95.2757037428
    # image[2,:,:]=image[2,:,:]-104.877445883
    # input_tensor = image.cuda()
    # input_tensor = get_transform()(image).unsqueeze(0).to(device)
    input_tensor = get_transform()(image).to(device)
    with torch.no_grad():
        output = model(input_tensor.unsqueeze(0))
    return output

def evaluate(gt_dir, img_dir=None, model=None, pred_dir=None, device='cpu'):
    mae, mse, total = 0.0, 0.0, 0
    gt_files = sorted([f for f in os.listdir(gt_dir) if f.endswith('.h5')])

    for fname in tqdm(gt_files, desc="Evaluating"):
        gt_path = os.path.join(gt_dir, fname)
        with h5py.File(gt_path, 'r') as f:
            gt_density = np.asarray(f['density'])
        # gt_count = gt_density.sum()
        gt_count = np.sum(gt_density)

        if model and img_dir:
            img_path = os.path.join(img_dir, fname.replace('.h5', '.jpg').replace('GT_', ''))
            pred_density = predict_density(model, img_path, device)
        elif pred_dir:
            pred_path = os.path.join(pred_dir, fname)
            if not os.path.exists(pred_path):
                print(f"Missing prediction for {fname}, skipping.")
                continue
            with h5py.File(pred_path, 'r') as f:
                pred_density = np.asarray(f['density'])
        else:
            raise ValueError("Either model+img_dir or pred_dir must be provided.")

        pred_count = pred_density.detach().cpu().sum().numpy()
        # print(f"GT {gt_count:.2f}\t Predict {pred_count:.2f}")
        
        error = abs(pred_count - gt_count)
        mae += error
        mse += error ** 2
        total += 1

    mae /= total
    rmse = (mse / total) ** 0.5
    # rmse = (mse / total)

    print(f"\nEvaluation Results:")
    print(f"  MAE  = {mae:.2f}")
    print(f"  RMSE = {rmse:.2f}")

def main():
    part = PART.upper()
    root = os.path.abspath("dataset")
    gt_dir = os.path.join(root, f"part_{part}", "test_data", "ground-truth")
    img_dir = os.path.join(root, f"part_{part}", "test_data", "images")

    if not os.path.isdir(gt_dir):
        raise FileNotFoundError(f"Ground truth directory not found: {gt_dir}")

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    if MODEL:
        model = load_model(MODEL, device)
        evaluate(gt_dir, img_dir=img_dir, model=model, device=device)
    elif PREDDIR:
        pred_dir = os.path.abspath(PREDDIR)
        evaluate(gt_dir, pred_dir=pred_dir)
    else:
        raise ValueError("You must provide either --model or --pred_dir")

if __name__ == '__main__':
    main()


Evaluating: 100%|██████████| 182/182 [00:03<00:00, 54.08it/s]


Evaluation Results:
  MAE  = 169.73
  RMSE = 278.66
